<!-- # CNN autoencoder and Clustering from MTRX data

Use this notebook to load Scienta Omicron Matrix format SPM data and create standardised images for machine learning training and analysis. The code can generate both JPG image data, useful for manually checking the data, and windowed numpy data that can be loaded into ML models. 

The notebook then creates an autoencoder for training on a large dataset, followed by KMEANS clustering. 

**Author**: Steven R. Schofield  
**Created**: November, 2024 -->

# Machine learning autoencoder + Kmeans for STM image data analysis
## Steven R. Schofield (Universtiy College London Dec. 2024)

### Load required modules

In [2]:
# Define path where to find the module. This allows for a different path depending on where the code is running (my mac or the cluster)
import os

module_path_list = [
    '/Users/steven/academic-iCloud/Python/modules',
    '/hpc/srs/Python/modules'
] 

data_path_list = [
    '/Users/steven/Python-data',
    '/hpc/srs/Python-data'
]

module_path = next((p for p in module_path_list if os.path.exists(p)), None)
if not module_path:
    exit("No valid module paths.")
else:
    print('module_path = {}'.format(module_path))

data_path = next((p for p in data_path_list if os.path.exists(p)), None)
if not module_path:
    exit("No valid data paths.")
else:
    print('data_path = {}'.format(data_path))

# adjust tensorflow output level
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' # 0 default all messages, 1 warnings and errors, 2, errors, 3 fatal errors only

module_path = /Users/steven/academic-iCloud/Python/modules
data_path = /Users/steven/Python-data


In [3]:
# # Ensure modules are reloaded 
%load_ext autoreload
%autoreload 2

# Import standard modules
import os, sys
import numpy as np

import platform

from datetime import datetime

# Add custom module path to list
sys.path.append(module_path)

# Import custom module
import SRSML24.data_prep as dp
import SRSML24.model as m
import SRSML24.utils as ut

import tensorflow as tf
#from tensorflow.keras.optimizers.legacy import Adam 
from tensorflow.keras.optimizers import Adam

import matplotlib.pyplot as plt

#import platform 

m.print_system_info()

start_time = dp.current_datetime()


Python version: 3.8.18 (default, Sep 11 2023, 08:17:16) 
[Clang 14.0.6 ]
TensorFlow version: 2.13.0
TensorFlow is built with CUDA: False
TensorFlow is built with ROCm: False

System: Darwin 24.1.0 (arm64)
Platform: macOS-15.1.1-arm64-arm-64bit
Processor: arm

Number of GPUs available to TensorFlow: 1
GPU Device: PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')

>>> Running with GPU available <<<  (macOS-15.1.1-arm64-arm-64bit)

Current time 2025-01-03 18:04:19


### Programme variable setup

In [4]:
# Parameters for windows creation
job_name = 'MSc2024_Data'
job_data_path = dp.create_new_data_path(data_path, job_name, include_date=False)
mtrx_train_path = os.path.join(data_path, 'mtrx/train')
mtrx_test_path = os.path.join(data_path, 'mtrx/test')
mtrx_predict_path = os.path.join(data_path, 'mtrx/predict')
flatten_method = 'poly_xy'
pixel_density = 15.0        # Convert all images to a constant pixel density
pixel_ratio = 0.7           # If an image has less than this % in the slow scan direction it is discarded
data_scaling = 1.e9         # Scale the z-height of the data
window_size = 16            # Window size for training/validation
window_pitch = 8            # Window pitch for training/validation
together = True             # Set this True to save image windows for a mtrx image as a single file rather than separate files.
collate = False             # Set this True to remove all subfolder directories and save all data in root data path
save_window_jpgs = False    # Save the windows as jpgs for inspection
save_windows = True         # Save the windows as numpy files
save_jpg = False            # Save the full image as a jpg
verbose = False             # Set this True to print out more information

# Parameters for training
model_name = 'unet_' + job_name
batch_size = 128
buffer_size = 12800 # shuffling
learning_rate = 1e-4
epochs = 5

# Parameters for clustering
cluster_model_name = model_name + '_kmeans'
cluster_batch_size = 5120 # This is the number of latent features in a batch for clustering. 
                          # Does not have to be the same as for training and probably should 
                          # be larger. 
cluster_buffer_size = cluster_batch_size * 5    # shuffling buffer
num_clusters=20                                 # Desired number of clusters (centroids) to form in the data.
n_init=50                                       # Number of times the algorithm will run with different centroid seeds.
max_iter=1000                                   # Maximum iterations allowed for each mini-batch to refine centroids.
reassignment_ratio=0.05                         # Fraction of clusters reassigned per step; lower values stabilize updates.

# Parameters for PREDICTIONS
predict_window_pitch = 4                        # Window pitch for prediction

# DATA LIMITS FOR TESTING THE CODE
mtrx_train_data_limit = 200 #None                    # Number of MTRX files to process (training)
mtrx_test_data_limit = 20 #None                     # Number of MTRX files to process (validation)

train_data_limit = None                         # Limit the data used in the autoencoder training
test_data_limit = None                          # Limit the data used in the autoencoder training (validation)

### Process Matrix format data to windows for autoencoder training and validation

In [4]:
# # REMOVE ALL DATA FOLDERS EXCEPT MTRX 
# dp.delete_data_folders(job_data_path, subdirectories=["jpg", "windows", "windows-jpg"], override=True)

In [5]:
# # Training data
# mtrx_train_file_list, _ = dp.list_files_by_extension(mtrx_train_path,'Z_mtrx',verbose=False)

# dp.process_mtrx_files(
#     mtrx_train_file_list[0:mtrx_train_data_limit],
#     job_data_path, # save data path
#     flatten_method = flatten_method, pixel_density = pixel_density, pixel_ratio = pixel_ratio,
#     data_scaling = data_scaling, window_size = window_size, window_pitch = window_pitch,
#     save_windows = save_windows,
#     save_window_jpgs=save_window_jpgs,
#     save_jpg = save_jpg,
#     together = together,
#     collate = collate,
#     verbose = verbose
#     )

# # Test data
# mtrx_test_file_list, _ = dp.list_files_by_extension(mtrx_test_path,'Z_mtrx',verbose=False)

# dp.process_mtrx_files(
#     mtrx_test_file_list[0:mtrx_test_data_limit],
#     job_data_path, # save data path
#     flatten_method = flatten_method, pixel_density = pixel_density, pixel_ratio = pixel_ratio,
#     data_scaling = data_scaling, window_size = window_size, window_pitch = window_pitch,
#     save_windows = save_windows,
#     save_window_jpgs=save_window_jpgs,
#     save_jpg = save_jpg,
#     together = together,
#     collate = collate,
#     verbose = verbose
#     )

### Build tensorflow data pipeline for training and validation of autoencoder

In [6]:
# Training data - tensorflow data pipeline for autoencoder
windows_train_path = os.path.join(job_data_path, 'windows/train')
train_files, num_train = dp.list_files_by_extension(windows_train_path, 'npy')
train_files = train_files[:train_data_limit]

# Create dataset with prefetching
train_dataset = m.create_tf_dataset_batched(
    train_files, 
    batch_size=batch_size, 
    buffer_size=buffer_size, 
    window_size=window_size,
    is_autoencoder=True, 
    shuffle=True)

# Validation data - tensorflow data pipeline for autoencoder
windows_test_path = os.path.join(job_data_path, 'windows/test')
test_files, num_test = dp.list_files_by_extension(windows_test_path, 'npy')
test_files = test_files[:test_data_limit]

# Create dataset with prefetching
test_dataset = m.create_tf_dataset_batched(
    test_files, 
    batch_size=batch_size, 
    buffer_size=buffer_size, 
    window_size=window_size,
    is_autoencoder=True, 
    shuffle=True)

Found 217 files with extension 'npy' in directory:
/Users/steven/Python-data/MSc2024_Data/windows/train
Data pipeline created with 217 files, batch size: 128, window size: 16
Sample batch shape: (128, 16, 16, 1)

Found 2 files with extension 'npy' in directory:
/Users/steven/Python-data/MSc2024_Data/windows/test
Data pipeline created with 2 files, batch size: 128, window size: 16
Sample batch shape: (128, 16, 16, 1)



In [7]:
# # Build and compile the UNET model
# autoencoder_model = m.build_autoencoder(window_size=window_size,model_name=model_name)
# autoencoder_model.summary()

# # Check if running on Apple Silicon
# is_mac_silicon = platform.system() == "Darwin" and platform.processor() == "arm"

# if is_mac_silicon:
#     print("Detected Mac with Apple Silicon. Compiling the model with the legacy RMSprop optimizer for compatibility with TensorFlow-metal.")
#     autoencoder_model.compile(
#         optimizer=tf.keras.optimizers.legacy.RMSprop(learning_rate=learning_rate),
#         loss='mean_squared_error',
#         metrics=['mse', 'mae']
#     )
# else:
#     print("Compiling the model with the RMSprop optimizer.")
#     autoencoder_model.compile(
#         optimizer=tf.keras.optimizers.RMSprop(learning_rate=learning_rate),
#         loss='mean_squared_error',
#         metrics=['mse', 'mae']
#     )

### Train autoencoder

In [8]:
# # Train the model using the tf.data datasets
# history = autoencoder_model.fit(
#     train_dataset,
#     validation_data=test_dataset,
#     epochs=epochs,
#     shuffle=True,
#     verbose=1
# )
# model_train_time = datetime.now().strftime('%Y%m%d_%H%M%S')

# # Save the model as soon as training completes
# model_path = os.path.join(job_data_path,'model')
# m.save_model(autoencoder_model, model_path, model_name=model_name, model_train_time=model_train_time)

# end_time = dp.current_datetime()
# dp.elapsed_time(start_time,end_time)

In [9]:
# # Plot training history
# m.plot_training_history(history, 
#                         loss_name='loss', 
#                         val_loss_name='val_loss', 
#                         metric_names=['mse', 'mae'], 
#                         save_to_disk=True,
#                         model_name=model_name, 
#                         model_train_time=model_train_time,
#                         val_metric_names=['val_mse', 'val_mae'],
#                         output_path=model_path,
#                         dpi=150)

### Extract Latent Features

In [10]:
# dp.delete_data_folders(
#     job_data_path, 
#     subdirectories='latent_features',
#     override=True)

In [11]:
# Load the trained autoencoder model
model_path = os.path.join(job_data_path,'model')
autoencoder_model = m.load_model(model_path, model_name=model_name)

Model loaded from: /Users/steven/Python-data/MSc2024_Data/model/unet_MSc2024_Data.keras


In [12]:
# Training data - tensorflow data pipeline 
windows_train_path = os.path.join(job_data_path, 'windows/train')
train_files, num_train = dp.list_files_by_extension(windows_train_path, 'npy')
train_files = train_files[:train_data_limit]

train_dataset = m.create_tf_dataset_batched(
    train_files, 
    batch_size=cluster_batch_size, 
    buffer_size=cluster_buffer_size, 
    window_size=window_size,
    is_autoencoder=True, 
    shuffle=True)

Found 217 files with extension 'npy' in directory:
/Users/steven/Python-data/MSc2024_Data/windows/train
Data pipeline created with 217 files, batch size: 5120, window size: 16
Sample batch shape: (5120, 16, 16, 1)



In [13]:
# # Path to save latent features to disk
# latent_features_path = os.path.join(job_data_path, 'latent_features')

# m.extract_latent_features_to_disk_from_prebatched_windows(
#     autoencoder_model, 
#     train_dataset, 
#     latent_features_path, 
#     features_name='latent_features_train',
#     return_array=False,
#     verbose=False)

### Train KMEANS using latent features saved to disk

In [14]:
# List and sort latent feature files
latent_features_path = os.path.join(job_data_path, 'latent_features')
latent_features_files, num_latent_files = dp.list_files_by_extension(latent_features_path, 'npy')

Found 159 files with extension 'npy' in directory:
/Users/steven/Python-data/MSc2024_Data/latent_features


In [15]:
# Load the latent features from disk into a tensor dataset pipeline
latent_features_dataset = m.create_latent_features_tf_dataset(
    latent_features_files,
    batch_size=cluster_batch_size,
    shuffle=True, 
    shuffle_buffer_size=cluster_buffer_size)

Data pipeline created with 159 files, batch size: 5120
Shuffling enabled with buffer size: 25600
Batch shape: (5120, 1024)


In [16]:
cluster_model, convergence_history = m.train_kmeans(
    latent_features_dataset,                # tf.data.Dataset containing batches of latent feature vectors.
    batch_size=cluster_batch_size,          # Size of each batch for the KMeans model (controls memory usage and stability).
    num_clusters=num_clusters,              # Desired number of clusters (centroids) to form in the data.
    n_init=n_init,                          # Number of times the algorithm will run with different centroid seeds.
    max_iter=max_iter,                      # Maximum iterations allowed for each mini-batch to refine centroids.
    reassignment_ratio=reassignment_ratio   # Fraction of clusters reassigned per step; lower values stabilize updates.
)

# Save cluster model
cluster_model_path = os.path.join(job_data_path,'cluster_model')
m.save_cluster_model(cluster_model, cluster_model_path, model_name=cluster_model_name)

/opt/anaconda3/envs/tf/lib/python3.8/site-packages/sklearn/cluster/_kmeans.py:1930: RuntimeWarning: Explicit initial center position passed: performing only one init in MiniBatchKMeans instead of n_init=50.
  super()._check_params_vs_input(X, default_n_init=3)


Batch 1 processed. Inertia: 170.38534545898438
Batch 2 processed. Inertia: 167.76954650878906
Batch 3 processed. Inertia: 172.38441467285156
Batch 4 processed. Inertia: 166.822509765625
Batch 5 processed. Inertia: 174.10671997070312
Batch 6 processed. Inertia: 167.84454345703125
Batch 7 processed. Inertia: 168.73667907714844
Batch 8 processed. Inertia: 172.84779357910156
Batch 9 processed. Inertia: 173.752197265625
Batch 10 processed. Inertia: 170.63973999023438
Batch 11 processed. Inertia: 170.47056579589844
Batch 12 processed. Inertia: 169.163818359375
Batch 13 processed. Inertia: 170.18507385253906
Batch 14 processed. Inertia: 174.143310546875
Batch 15 processed. Inertia: 171.4431610107422
Batch 16 processed. Inertia: 170.62582397460938
Batch 17 processed. Inertia: 167.7606964111328
Batch 18 processed. Inertia: 165.46514892578125
Batch 19 processed. Inertia: 170.24818420410156
Batch 20 processed. Inertia: 173.92453002929688
Batch 21 processed. Inertia: 167.010498046875
Batch 22 proc

### Process Matrix format data to windows for making predictions 

In [17]:
# dp.delete_data_folders(
#     job_data_path, 
#     subdirectories=['windows/predict','windows-jpg/predict','jpg/predict'],
#     override=True)

In [18]:
# # Prediction data in MTRX format
# mtrx_predict_file_list, _ = dp.list_files_by_extension(mtrx_predict_path,'Z_mtrx',verbose=False)

# dp.process_mtrx_files(
#     mtrx_predict_file_list,
#     job_data_path, # save data path
#     flatten_method = flatten_method, pixel_density = pixel_density, pixel_ratio = pixel_ratio,
#     data_scaling = data_scaling, window_size = window_size, 
#     window_pitch = predict_window_pitch,
#     save_windows = save_windows,
#     save_window_jpgs=save_window_jpgs,
#     save_jpg = save_jpg,
#     together = together,
#     collate = collate,
#     verbose = verbose
# )

In [19]:
# # Function to summarize programme parameters using pandas
# import pandas as pd
# def summarize_programme_parameters_pandas():
#     # Create a dictionary for all parameters
#     parameters = {
#         "Parameter": [
#             "Job Name", 
#             "Flatten Method", "Pixel Density", "Pixel Ratio", "Data Scaling", "Window Size",
#             "Window Pitch", "Save as Single File", "Collate in Root Directory", "Model Name",
#             "Batch Size", "Buffer Size", "Learning Rate", "Epochs", "Cluster Model Name",
#             "Cluster Batch Size", "Cluster Buffer Size", "Number of Clusters",
#             "Initialization Attempts", "Max Iterations", "Reassignment Ratio",
#             "Prediction Window Pitch", "Training Data Limit", "Testing Data Limit",
#             "Matrix Training Data Limit", "Matrix Testing Data Limit"
#         ],
#         "Value": [
#             job_name, 
#             flatten_method, pixel_density, pixel_ratio, data_scaling, window_size,
#             window_pitch, together, collate, model_name+'_'+model_train_time, batch_size, buffer_size,
#             learning_rate, epochs, cluster_model_name, cluster_batch_size, cluster_buffer_size,
#             num_clusters, n_init, max_iter, reassignment_ratio, predict_window_pitch,
#             train_data_limit, test_data_limit, mtrx_train_data_limit, mtrx_test_data_limit
#         ]
#     }

#     # Create a DataFrame from the dictionary
#     df = pd.DataFrame(parameters)

#     # Add a title above the DataFrame
#     from IPython.display import display, Markdown
#     display(Markdown("## Parameters Summary - {}".format(model_train_time)))
#     display(df)

# # Call the function to display the summary
# summarize_programme_parameters_pandas()


### Make predictions using the trained autoencoder and KMEANS models

In [6]:
# Load the trained autoencoder
model_path = os.path.join(job_data_path,'model')
autoencoder_model = m.load_model(model_path, model_name=model_name)

# Load a previously saved cluster model from disk
cluster_model_path = os.path.join(job_data_path,'cluster_model')
cluster_model = m.load_cluster_model(cluster_model_path, model_name=cluster_model_name)

Model loaded from: /Users/steven/Python-data/MSc2024_Data/model/unet_MSc2024_Data.keras
Cluster model loaded from: /Users/steven/Python-data/MSc2024_Data/cluster_model/unet_MSc2024_Data_kmeans.pkl


In [ ]:
# Folder where image prediction windows are stored. 
windows_predict_path = os.path.join(job_data_path, 'windows/predict')
# Get the list of image windows files to make predictions on
predict_data_files_list, predict_data_files_num = dp.list_files_by_extension(windows_predict_path,'.npy',verbose=False)
# Get the corresponding image coordimages list file
image_windows_coordinates_file_list , _ = dp.list_files_by_extension(windows_predict_path,'.txt',verbose=False)
image_windows_coordinates_file_list = [
    name for name in image_windows_coordinates_file_list 
    if "coordinates" in name
]

# Folder to save latent_feature_predictions
predict_latent_features_path = os.path.join(job_data_path, 'latent_features_predictions')

# %matplotlib inline
# %matplotlib auto


# Make predictions on the image windows and save the latent features to disk
for prediction_file, coords_file in zip(predict_data_files_list,image_windows_coordinates_file_list):
    # Load the windows for the image as a numpy file
    image_windows = np.load(prediction_file)
    # Load the image window coordinates
    image_windows_coordinates = dp.load_coordinates_file(coords_file)
    # Reconstruct the original image from the loaded image windows
    reconstructed_img = dp.reconstruct_image(image_windows,image_windows_coordinates,window_size)
    
    # Make a tensorflow data pipeline of just the image windows for this image.
    num_windows = image_windows.shape[0]
    print('\n---\nProcessing file {}'.format(os.path.basename(prediction_file)))
    predict_dataset = m.create_tf_dataset_batched(
        [prediction_file], 
        batch_size=num_windows, 
        buffer_size=num_windows, 
        window_size=window_size,
        is_autoencoder=False, 
        shuffle=False)
    # make the latent features for each window using the autoencoder model 
    latent_predict_features, num_latent_predictions = m.extract_latent_features_to_disk_from_prebatched_windows(
        autoencoder_model, 
        predict_dataset, 
        '',                 # we are not saving these predictions to disk so don't need a folder or name
        features_name='',
        return_array=True,
        verbose=False)
    # make preductions 
    cluster_predictions = cluster_model.predict(latent_predict_features)
    # Build the reconstruction of the predicted cluster label data
    cluster_img = dp.reconstruct_cluster_image(image_windows_coordinates,window_size, cluster_predictions)
    # Pad the cluster image to the original image size
    cluster_img = ut.padded_cluster_img = ut.pad_cluster_image(reconstructed_img,cluster_img,window_size)
    image_name = os.path.splitext(os.path.basename(prediction_file))[0]
    # Path to save latent features to disk
    predictions_path = os.path.join(job_data_path, 'predictions')
    m.display_reconstructed_and_cluster_images(reconstructed_img,cluster_img,show_overlay=True,
                                                save_to_disk=False,
                                                #predictions_time_stamp=model_train_time,
                                                output_path=job_data_path,
                                                image_name=image_name,
                                                dpi=150)
    break
    

Found 32 files with extension '.npy' in directory:
/Users/steven/Python-data/MSc2024_Data/windows/predict
Found 64 files with extension '.txt' in directory:
/Users/steven/Python-data/MSc2024_Data/windows/predict
Using matplotlib backend: MacOSX

---
Processing file default_2011Feb11-105510_STM-STM_Spectroscopy--110_16_BD_all_windows.npy
Not shuffling
Data pipeline created with 1 files, batch size: 49284, window size: 16
Sample batch shape: (49284, 16, 16, 1)
.Combined latent features shape: (49284, 1024)


In [7]:
# data = cluster_img

# # Function to display pixel values on click
# def on_click(event):
#     # Check if the click is on the image
#     if event.inaxes:
#         # Get the row and column indices
#         col, row = int(event.xdata + 0.5), int(event.ydata + 0.5)
#         # Get the pixel value
#         pixel_value = data[row, col]
#         print(f"Clicked on ({row}, {col}) with value: {pixel_value}")

# # Plot the image
# fig, ax = plt.subplots(figsize=(10, 8))  # Adjust figsize (width, height) as desired
# cax = ax.imshow(data, cmap='viridis', interpolation='nearest')
# fig.colorbar(cax, ax=ax)
# ax.set_title("Click on the image to get pixel value")

# # Connect the click event to the function
# cid = fig.canvas.mpl_connect('button_press_event', on_click)

# # Show the plot
# plt.show()


2025-01-03 17:59:59.662 python[90058:5249419] +[IMKClient subclass]: chose IMKClient_Modern
2025-01-03 17:59:59.662 python[90058:5249419] +[IMKInputSession subclass]: chose IMKInputSession_Modern


Clicked on (444, 618) with value: 5.0
Clicked on (513, 439) with value: 5.0
Clicked on (499, 347) with value: 8.0


2025-01-03 18:00:10.641 python[90058:5249419] Warning: Window move completed without beginning


Clicked on (428, 70) with value: 19.0
Clicked on (776, 143) with value: 19.0
Clicked on (496, 224) with value: 10.0
Clicked on (379, 266) with value: 19.0
Clicked on (168, 203) with value: 18.0
Clicked on (151, 243) with value: 19.0
Clicked on (157, 199) with value: 11.0


In [8]:
for i in range(int(np.max(cluster_img))):
    plt.imshow(cluster_img==i, cmap='viridis')
    plt.show()